In [1]:
%load_ext autoreload
%autoreload 2

import logging
from dotenv import load_dotenv
load_dotenv()

logging.basicConfig(level=logging.INFO)

logger = logging.getLogger(__name__)

In [2]:
from projects import set_project_root_for_notebook
set_project_root_for_notebook()

INFO:src.environment:Target: DEV
INFO:src.environment:Developer: FDDIFERD
INFO:src.environment:Schema Name: DEV_FDDIFERD
INFO:projects:Current working directory: /Users/donatodiferdinando/Documents/GitHub/ml-snowflake-template


In [3]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from projects.core.chargebacks_disputes_analysis.main import Service

svc = Service(use_cache=False)
date = (datetime.now() - relativedelta(months=6)).strftime("%Y-%m-%d")
svc.set_df(date)

INFO:snowflake.snowpark:AST state has not been set explicitly. Defaulting to ast_enabled = True.
INFO:projects.core.chargebacks_disputes_analysis.main:Querying data from snowflake
INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 3.18.0, Python Version: 3.10.18, Platform: macOS-26.1-arm64-arm-64bit
INFO:snowflake.connector.connection:Connecting to GLOBAL Snowflake domain
INFO:snowflake.snowpark.session:Snowpark Session information: 
"version" : 1.42.0,
"python.version" : 3.10.18,
"python.connector.version" : 3.18.0,
"python.connector.session.id" : 23301362205414,
"os.name" : Darwin

INFO:src.connection.session:Created or replaced database ML_LAYER_DB and schema DEV_FDDIFERD
INFO:src.connection.session:Connected to Snowflake for project CORE
INFO:src.connection.session:  Account: "vx70256-sw09563"
INFO:src.connection.session:  User: "ML_LAYER_USER"
INFO:src.connection.session:  Role: "ML_LAYER_ROLE"
INFO:src.connection.session:  Database: "ML_LAYER_DB"
INFO:src

,REF_TXN_ID,TXN_ID,PROCESSOR,TYPE,MERCHANT_ACCOUNT,CARD_NETWORK,BRAND_SLUG,SKU_TYPE_SLUG,CHANNEL_SLUG,TRAFFIC_SOURCE_SHORT_NAME,BIN,OFFER_TYPE,COUNT
0,TXN_1432099234,1.432099e+09,adyen,fraud,InteliusECOM,visa,Intelius,ReversePhoneLookup,Affiliate,2X79,434769,trial,1
1,TXN_1438900954,1.438901e+09,adyen,dispute,TruthFinderECOM,pulse,TruthFinder,Plus,PaidSearch,GOOGSRCH,414314,standard,0
2,TXN_1440716326,1.440716e+09,adyen,fraud,TruthFinderECOM,visa,TruthFinder,Background,EngagementEmail,EMAIL3,440066,standard,1
3,TXN_1432500883,1.432501e+09,adyen,dispute,TruthFinderECOM,visa,TruthFinder,Background,DisplayPartners,FPS,434769,standard,1
4,TXN_1396979833,1.396980e+09,adyen,fraud,InteliusECOM,mastercard,Intelius,Background,PaidSearch,PUBREC,518941,trial,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4370529,amzneks4,NaN,braintree,settlement,Truthfinder_instant_2,None,None,None,None,None,None,None,1
4370530,4h161yda,NaN,braintree,settlement,Truthfinder_instant_2,None,None,None,None,None,None,None,1
4370531,7saagkq3,NaN,braintree,settlement,Truthfinder_instant_2,None,None,None,None,None,None,None,1
4370532,c6r96nqc,NaN,braintree,settlement,Truthfinder_instant_2,None,None,None,None,None,None,None,1


In [4]:
from projects.core.chargebacks_disputes_analysis.main import Level

level = [
    Level.PROCESSOR,
    Level.CARD_NETWORK, 
    # Level.BRAND
]

svc.get_ratio_df(level)

INFO:projects.core.chargebacks_disputes_analysis.main:Computing ratio for [<Level.PROCESSOR: 'PROCESSOR'>, <Level.CARD_NETWORK: 'CARD_NETWORK'>]


,PROCESSOR,CARD_NETWORK,FRAUD_DISPUTES_COUNT,SETTLED_COUNT,RATIO,GLOBAL_RATIO,DELTA,DELTA_PERCENTAGE
2,adyen,discover,4,0,inf,0.012208,inf,inf
3,adyen,mastercard,15586,287155,0.054277,0.012208,0.042069,4.445932
8,adyen,visadankort,2,70,0.028571,0.012208,0.016363,2.340327
7,adyen,visa,21495,758279,0.028347,0.012208,0.016139,2.321950
13,braintree,mastercard,8171,987860,0.008271,0.012208,-0.003937,0.677523
12,braintree,discover,625,89481,0.006985,0.012208,-0.005224,0.572129
17,braintree,visa,20238,3065280,0.006602,0.012208,-0.005606,0.540807
1,adyen,cup,0,28,0.000000,0.012208,-0.012208,0.000000
9,braintree,accel,0,750,0.000000,0.012208,-0.012208,0.000000
10,braintree,americanexpress,0,104104,0.000000,0.012208,-0.012208,0.000000
